<a href="https://colab.research.google.com/github/anirbanghoshsbi/.github.io/blob/master/work/stock/momentum_shortlist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install pandas_ta --q

In [10]:
!pip install yfinance==0.2.43 --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.6/84.6 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.46
    Uninstalling yfinance-0.2.46:
      Successfully uninstalled yfinance-0.2.46


In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime as dt
import pandas_ta as ta
from scipy.stats import linregress
import warnings
warnings.filterwarnings('ignore')

# Define functions for each evaluation criterion
def calculate_moving_averages(data):
    data['50_MA'] = data['Close'].rolling(window=50).mean()
    data['200_MA'] = data['Close'].rolling(window=200).mean()
    return data

def calculate_rsi(data):
    data['RSI'] = ta.rsi(data['Close'], length=14)
    return data

def calculate_macd(data):
    macd = ta.macd(data['Close'])
    data['MACD'] = macd['MACD_12_26_9']
    data['MACD_Signal'] = macd['MACDs_12_26_9']
    return data

def calculate_bollinger_bands(data):
    data['BB_upper'] = data['Close'].rolling(window=20).mean() + 2 * data['Close'].rolling(window=20).std()
    data['BB_lower'] = data['Close'].rolling(window=20).mean() - 2 * data['Close'].rolling(window=20).std()
    return data

def check_uptrend(data, window=20):
    indices = np.arange(window)
    closes = data['Close'][-window:]
    slope, _, _, _, _ = linregress(indices, closes)
    return slope > 0

def evaluate_stock(ticker, data, nifty):
    stock_data = data[ticker]
    if len(stock_data) < 200:
        print(f"Not enough data for {ticker}. Skipping...")
        return None

    stock_data = calculate_moving_averages(stock_data)
    stock_data = calculate_rsi(stock_data)
    stock_data = calculate_macd(stock_data)
    stock_data = calculate_bollinger_bands(stock_data)

    # Ensure indices align
    stock_data = stock_data.dropna()
    if len(stock_data) == 0:
        print(f"No valid data after calculations for {ticker}.")
        return None

    # Calculate returns
    periods = [5, 21, 63]  # Weekly, Monthly, Quarterly
    returns = {}
    for period in periods:
        if len(stock_data) >= period:
            returns[period] = (stock_data['Close'][-1] - stock_data['Close'][-period]) / stock_data['Close'][-period] * 100
        else:
            returns[period] = np.nan

    # Benchmark return
    benchmark_return = (nifty['Close'][-1] - nifty['Close'][-63]) / nifty['Close'][-63] * 100 if len(nifty) >= 63 else np.nan

    # Initialize criteria
    criteria = {
        'Ticker': ticker,
        'Stock Return': returns.get(63, np.nan),
        'Benchmark Return': benchmark_return,
        'Outperformance': returns.get(63, 0) > benchmark_return,#*1.5,
        'Weekly Price Increase': returns.get(5, 0) > 0,
        'Monthly Price Increase': returns.get(21, 0) > 0,
        'Quarterly Price Increase': returns.get(63, 0) > 0,
        '50_MA above 200_MA (Golden Cross)': stock_data['50_MA'][-1] > stock_data['200_MA'][-1],
        'Price above 50_MA': stock_data['Close'][-1] > stock_data['50_MA'][-1],
        'Price above 200_MA': stock_data['Close'][-1] > stock_data['200_MA'][-1],
        'RSI between 50 and 70': 50 < stock_data['RSI'][-1] < 70,
        'Avg Volume Week > Month': stock_data['Volume'][-5:].mean() > stock_data['Volume'][-21:].mean(),
        'MACD above Signal Line': stock_data['MACD'][-1] > stock_data['MACD_Signal'][-1],
        'Price above Upper Bollinger Band': stock_data['Close'][-1] > stock_data['BB_upper'][-1],
        'Uptrend': check_uptrend(stock_data)
    }

    # Define weights
    weights = {
        'Outperformance': 3,
        'Weekly Price Increase': 1,
        'Monthly Price Increase': 2,
        'Quarterly Price Increase': 2,
        '50_MA above 200_MA (Golden Cross)': 1,
        'Price above 50_MA': 1,
        'Price above 200_MA': 1,
        'RSI between 50 and 70': 0.5,
        'Avg Volume Week > Month': 1,
        'MACD above Signal Line': 1,
        'Price above Upper Bollinger Band': 0.5,
        'Uptrend': 3
    }

    # Calculate total score
    total_score = sum(weights[key] * int(criteria[key]) for key in weights)
    criteria['Total Score'] = total_score

    return criteria

# Main code
end_date = dt.datetime.now()
start_date = end_date - dt.timedelta(days=365*2)  # 1 year of data

# Download benchmark data
nifty_index_symbol = "^nsei"  # Verify the correct symbol
nifty = yf.download(nifty_index_symbol, start=start_date, end=end_date)

# List of stock tickers to evaluate
nifty100_url = "https://raw.githubusercontent.com/anirbanghoshsbi/data/main/ind_nifty50list.csv"
nifty100 = pd.read_csv(nifty100_url)
stock_list =nifty100['Symbol'].apply(lambda x: x + ".NS").tolist()

# Download all stock data at once
stock_data = yf.download(stock_list, start=start_date, end=end_date, group_by='ticker')

# Evaluate stocks
results = []
for stock in stock_list:
    criteria = evaluate_stock(stock, stock_data, nifty)
    if criteria:
        results.append(criteria)

# Convert the results into a pandas DataFrame
df_results = pd.DataFrame(results)

# Set the Ticker as the index
df_results.set_index('Ticker', inplace=True)

# Sort the DataFrame based on Total Score
df_results.sort_values('Total Score', ascending=False, inplace=True)

# Save results to CSV
df_results.to_csv('stock_evaluation_results.csv')

# Display the top 10 stocks
print(df_results.head(10))


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  50 of 50 completed


               Stock Return  Benchmark Return  Outperformance  \
Ticker                                                          
TECHM.NS          12.626933          -1.75338            True   
WIPRO.NS           4.211135          -1.75338            True   
DIVISLAB.NS       19.748044          -1.75338            True   
HCLTECH.NS        12.813774          -1.75338            True   
HDFCBANK.NS        8.126564          -1.75338            True   
HDFCLIFE.NS        1.635032          -1.75338            True   
BAJAJ-AUTO.NS      8.528475          -1.75338            True   
BAJFINANCE.NS      3.698957          -1.75338            True   
ICICIBANK.NS       3.773198          -1.75338            True   
BHARTIARTL.NS     10.832668          -1.75338            True   

               Weekly Price Increase  Monthly Price Increase  \
Ticker                                                         
TECHM.NS                        True                    True   
WIPRO.NS                   

In [3]:
#Replace True with 1 and False with 0
df_numeric = df_results.replace({True: 1, False: 0})

# Sum across the rows, excluding 'Stock Return' and 'Benchmark Return'
df_numeric['Row_Sum'] = df_numeric.drop(columns=['Stock Return', 'Benchmark Return']).sum(axis=1)


In [4]:
# Display the updated DataFrame
df_numeric.sort_values(by='Row_Sum', ascending=False)[['Row_Sum','Stock Return','Benchmark Return']].head(10)

,Row_Sum,Stock Return,Benchmark Return
Ticker,,,
TECHM.NS,27.5,12.626933,-1.75338
WIPRO.NS,25.5,4.211135,-1.75338
DIVISLAB.NS,21.5,19.748044,-1.75338
HCLTECH.NS,21.5,12.813774,-1.75338
HDFCBANK.NS,20.5,8.126564,-1.75338
HDFCLIFE.NS,15.0,1.635032,-1.75338
BAJAJ-AUTO.NS,15.0,8.528475,-1.75338
BAJFINANCE.NS,15.0,3.698957,-1.75338
ICICIBANK.NS,15.0,3.773198,-1.75338


In [5]:
df_numeric.sort_values(by='Row_Sum', ascending=False)[['Row_Sum','Stock Return','Benchmark Return']].tail(10)

,Row_Sum,Stock Return,Benchmark Return
Ticker,,,
ADANIPORTS.NS,2.0,-12.208717,-1.75338
LT.NS,2.0,-6.447186,-1.75338
MARUTI.NS,2.0,-7.106928,-1.75338
INDUSINDBK.NS,2.0,-8.821852,-1.75338
TCS.NS,2.0,-7.747534,-1.75338
TATAMOTORS.NS,2.0,-21.309133,-1.75338
TITAN.NS,2.0,-4.672095,-1.75338
RELIANCE.NS,0.0,-11.214193,-1.75338
TATASTEEL.NS,0.0,-8.348205,-1.75338
